In [33]:
import random
import pandas as pd
import numpy as np
import os
import glob
import warnings
warnings.filterwarnings('ignore')

In [213]:
def make_dataset(input_path, target_path):
    train = pd.DataFrame()
    all_input_list = sorted(glob.glob(input_path))
    all_target_list = sorted(glob.glob(target_path))
    for x, y in zip(all_input_list,all_target_list):
        x = pd.read_csv(x)
        y = pd.read_csv(y)
        x['obs_time'] = x['obs_time'].str[:2]
        x = x.iloc[:,1:]
        col_list = x.columns[1:]
        for i in range(0,28) :
            day = x.iloc[24*i:24*i+24]
            time_list = day['obs_time'].unique()
            if len(time_list) > 24 :
                for i in range(0,len(time_list)):
                    x['obs_time'] = x['obs_time'].replace(time_list[24+i],time_list[i])
            for col in col_list :
                for time in time_list :
                    value = day[day['obs_time']==time][col].iloc[0]
                    x[col+time] = value
            nx = x.iloc[:1,15:]
            ny = y.iloc[i:i+1].reset_index(drop=True)
            xy = pd.merge(nx,ny,left_index=True, right_index=True)
            train = pd.concat([train,xy]).reset_index(drop=True)
    return train

In [214]:
train = make_dataset('./train_input/*.csv','./train_target/*.csv')

In [220]:
train.to_csv('train.csv',index=False)

In [242]:
train

,내부온도관측치00,내부온도관측치01,내부온도관측치02,내부온도관측치03,내부온도관측치04,내부온도관측치05,내부온도관측치06,내부온도관측치07,내부온도관측치08,내부온도관측치09,...,일간누적총광량16,일간누적총광량17,일간누적총광량18,일간누적총광량19,일간누적총광량20,일간누적총광량21,일간누적총광량22,일간누적총광량23,DAT,predicted_weight_g
0,25.300000,25.680357,25.273333,25.355000,25.391667,25.375000,26.436667,26.960000,27.128334,27.366666,...,210911.3740,230142.8960,249374.4180,268605.9400,269376.2938,269376.2938,269376.2938,269376.2938,1,0.167719
1,25.806666,25.981034,25.583334,25.476667,25.431666,25.406666,27.105000,27.765000,28.025000,28.196667,...,211220.7840,230452.3060,249683.8280,268915.3500,270032.8895,270032.8895,270032.8895,270032.8895,2,0.181787
2,26.098334,26.313559,25.878333,25.794999,25.660001,25.601667,27.388333,28.068334,28.228333,27.823333,...,210727.0981,229958.6201,249190.1421,268421.6641,269399.5381,269399.5381,269399.5381,269399.5381,3,0.265921
3,25.438333,25.783051,25.321666,25.211667,25.275000,25.245000,26.893333,27.470000,26.641667,26.379661,...,211220.7840,230452.3060,249683.8280,268915.3500,270032.8895,270032.8895,270032.8895,270032.8895,4,0.423650
4,25.470000,25.462712,25.401667,25.331667,25.461667,25.410000,26.391666,26.698334,26.806666,26.938333,...,210838.9167,230070.4387,249301.9607,268533.4827,269511.3567,269511.3567,269511.3567,269511.3567,5,0.475272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,30.368333,30.362711,30.768333,30.645000,30.946666,30.826666,31.135000,31.525000,32.485000,30.155000,...,79507.0460,95162.5780,103428.9048,103438.1871,103438.1871,103438.1871,103438.1871,103438.1871,24,85.130577
668,23.680000,23.717241,23.818333,23.660000,23.658333,23.561666,23.730000,24.688333,25.783333,25.746667,...,155738.5910,171394.1230,179463.2992,179469.4874,179469.4874,179469.4874,179469.4874,179469.4874,25,93.772755
669,24.028333,24.075862,24.208333,24.101667,24.013333,23.900000,23.938333,24.725000,25.946667,25.996667,...,155505.1534,171160.6854,179423.2327,179432.5150,179432.5150,179432.5150,179432.5150,179432.5150,26,102.852332
670,24.050000,23.985000,24.058334,23.980000,23.966667,23.920000,23.950000,24.631667,25.600000,25.545000,...,155528.0655,171183.5975,179360.7245,179373.1009,179373.1009,179373.1009,179373.1009,179373.1009,27,107.513933


In [234]:
from sklearn.model_selection import train_test_split
X = train.iloc[:,:-1]
y = train.iloc[:,-1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [221]:
from sklearn.linear_model import LinearRegression

In [235]:
line_fitter = LinearRegression()
line_fitter.fit(X_train, y_train)

LinearRegression()

In [236]:
y_pred = line_fitter.predict(X_test)

In [237]:
from sklearn.metrics import mean_squared_error

RMSE = mean_squared_error(y_test, y_pred)**0.5

In [238]:
RMSE

96.76071513132946

In [239]:
y_train

,predicted_weight_g
60,1.968096
133,96.657944
30,1.333024
69,7.560952
292,66.197836
...,...
71,10.116180
106,85.195499
270,37.555938
435,92.670147


In [240]:
y_test

,predicted_weight_g
361,159.735773
158,22.879580
481,2.446409
639,53.130886
275,85.323501
...,...
165,36.790375
482,1.577447
667,85.130577
629,11.140989


In [241]:
y_pred

array([[ 1.38833370e+02],
       [ 4.59684663e+01],
       [ 1.54347138e-01],
       [ 4.92823378e+01],
       [ 6.04856883e+01],
       [ 1.50470642e+02],
       [-1.25763789e+00],
       [-2.73410942e+01],
       [ 3.17825779e+01],
       [ 1.16010973e+01],
       [ 3.99061350e+01],
       [ 1.58722118e+01],
       [ 1.06840432e+02],
       [ 6.22692578e+01],
       [ 4.57605693e+01],
       [ 1.30852798e+01],
       [-4.42991877e+02],
       [ 8.02393856e+01],
       [ 4.60051110e+01],
       [ 2.65191483e+01],
       [ 9.67826626e+01],
       [ 2.95959701e+01],
       [ 8.73803805e+01],
       [ 3.34883866e+01],
       [ 2.64374994e+01],
       [ 4.58175814e-01],
       [-1.18240097e+01],
       [ 7.07101344e+00],
       [ 8.90917048e+01],
       [ 3.51065984e+01],
       [ 3.82128432e+01],
       [ 1.67068532e+01],
       [ 5.94172316e+01],
       [ 6.00484015e+01],
       [ 1.41202671e+02],
       [ 4.88607264e+01],
       [ 2.93833234e+01],
       [ 2.34037668e+01],
       [ 2.1

In [243]:
import xgboost

In [244]:
xgb_model = xgboost.XGBRegressor()
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [245]:
y_pred = xgb_model.predict(X_test)

In [246]:
RMSE = mean_squared_error(y_test, y_pred)**0.5

In [247]:
RMSE

16.07830655137753

In [248]:
y_pred

array([154.81592  ,  21.989285 ,   1.9626219,  73.10609  ,  68.600426 ,
       158.94618  ,   3.2886705,   3.162233 ,  14.486697 ,   7.320275 ,
        62.323334 ,  13.741892 , 125.62292  ,  82.154045 ,  22.209543 ,
         4.1283436,  62.018665 ,   6.5844626,  53.417942 ,  17.503656 ,
        83.49432  ,  20.69621  , 146.83098  , 108.67321  ,  46.08571  ,
         1.5901636,   2.610946 ,   5.3459053, 153.12326  ,  31.149895 ,
        89.54431  ,   9.217942 ,  81.33305  , 104.70359  , 118.12083  ,
        22.062952 ,   8.142414 ,  43.819763 ,   4.991174 ,  23.70666  ,
        56.81187  ,  13.348988 ,   1.4953232,   1.0095584, 122.00635  ,
        50.004673 ,   8.56008  ,   6.6612687,  63.18482  ,   4.7665253,
         6.5126896,  45.33655  ,  38.801414 ,  11.184998 ,  28.300474 ,
        68.7368   ,  77.519455 ,   5.04485  , 116.12353  ,   9.990034 ,
        33.69527  ,  10.728735 ,   0.6876166,  33.19218  ,   2.6065593,
        75.44797  ,  15.703377 , 138.01538  ], dtype=float32)

In [250]:
y_test.values

array([[159.73577319],
       [ 22.87957972],
       [  2.44640894],
       [ 53.13088643],
       [ 85.32350088],
       [158.96935351],
       [  3.38077829],
       [  2.40926574],
       [ 18.40023037],
       [  6.22172345],
       [ 35.27163062],
       [ 14.24088199],
       [139.19354847],
       [ 76.79468143],
       [ 16.95740211],
       [  3.47504756],
       [ 56.77197545],
       [  6.47144847],
       [128.59818143],
       [ 14.64236382],
       [115.53952392],
       [ 18.97604436],
       [159.41315418],
       [121.69221035],
       [ 30.13595483],
       [  0.66107202],
       [  3.34409572],
       [  4.28353088],
       [159.28334705],
       [ 17.83987318],
       [116.67833333],
       [  8.10676356],
       [ 74.61694021],
       [114.89280874],
       [124.1212748 ],
       [ 42.36402599],
       [ 11.33468635],
       [ 34.58246516],
       [  6.23378222],
       [ 19.20997138],
       [ 10.01923041],
       [ 10.33965613],
       [  1.45350551],
       [  1